In [ ]:
import folium

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('Lima-intersecciones.csv')

In [ ]:
df.columns = df.columns.str.strip()

In [ ]:
df.head()

In [ ]:
subset_of_df = df.sample(n=500)

In [ ]:
some_map = folium.Map(location=[subset_of_df['Y'].mean(),subset_of_df['X'].mean()],zoom_start=100)

In [ ]:
for row in subset_of_df.itertuples():
  some_map.add_child(folium.Marker(location=[row.Y,row.X], popup=row.ID))

In [ ]:
some_map